This notebook process staging dataset to targeted fact table.

In [1]:
import pandas as pd
import numpy as np
import yaml
from glob import glob
import os
from pathlib import Path
from datetime import date, timedelta, datetime
from pandas.tseries.holiday import USFederalHolidayCalendar
from pathlib import Path
import re
import hashlib

### Import staging dataset

In [9]:
df_dim = pd.read_csv("preprocessed_ridership_output/dataset_metadata.csv")

In [10]:
df_dim.head()

,dataset_id,dataset_name,organization_name,service_name,start_date_collected,end_date_collected,reporting_unit,ridership_measure,geographic_grain,notes,route_id_exists,route_name_exists,direction_exists,stop_id_exists,stop_name_exists,stop_lat_exists,stop_lon_exists,boardings_exists,alightings_exists,ridership_exists
0,189FC69D989010FD,golden_gate_park_shuttle_ridership,City and County of San Francisco,Golden Gate Park Shuttle,2024-07-01,2025-06-30,day,daily,stop,Collected manually by operators. Weekday two v...,0,0,1,0,1,0,0,0,0,1
1,957BBF3AD8FC1B44,samtrans_ridership,San Mateo County Transit District,SamTrans,2025-08-01,2025-08-31,day,daily,trip_stop,APCs are not NTD-certified and has error rate ...,1,0,0,1,1,1,1,1,1,0
2,1B1BADA1E00153AF,sunline_transit_ridership,SunLine Transit Agency,SunLine Transit,2022-07-01,2025-06-30,fiscal_year,avg_daily,stop,NaN,0,0,0,1,1,1,1,1,1,0
3,581003114DDAFDBE,santa_cruz_metro_ridership,Santa Cruz Metropolitan Transit District,Santa Cruz METRO,2024-07-01,2025-06-30,fiscal_year,total,stop,Several buses without APC hardware (5 in March...,0,0,0,1,1,0,0,1,1,0
4,1CCF9080DC015EB8,foothill_transit_ridership,Foothill Transit,Foothill Transit,2024-07-01,2025-06-30,day,daily,trip_stop,NaN,1,0,1,1,0,1,1,1,1,0


In [11]:
df_ridership = pd.read_csv("staging_ridership_output/staging_ridership_2026-02-09.csv", dtype={"stop_id": "string",
                                                                                                  "gtfs_stop_id": "string"})
df_ridership.head()

/tmp/ipykernel_202/1979777153.py:1: DtypeWarning: Columns (4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ridership = pd.read_csv("staging_ridership_output/staging_ridership_2026-02-09.csv", dtype={"stop_id": "string",


,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,stop_lat,stop_lon,daily_boardings,daily_alightings,daily_total_ridership,day_type,daily_ridership_basis,start_date,end_date,gtfs_stop_id
0,FF27A73278C43BCE,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,45.0,weekday,reported_daily,2024-07-01,2024-07-01,7602
1,31AC9C20F68713A9,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,59.0,weekday,reported_daily,2024-07-02,2024-07-02,7602
2,E021D40119E1054C,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,74.0,weekday,reported_daily,2024-07-03,2024-07-03,7602
3,B1F06206C22A5484,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,177.0,weekday,reported_daily,2024-07-04,2024-07-04,7602
4,5C7966570540D9F0,189FC69D989010FD,City and County of San Francisco,Golden Gate Park Shuttle,NaN,NaN,NaN,<NA>,Blue Heron Boathouse,37.77084,-122.476829,NaN,NaN,110.0,weekday,reported_daily,2024-07-05,2024-07-05,7602


In [12]:
# take stop id provided by agency by default, if no stop id provided, use gtfs stop id
df_ridership["stop_id_comb"] = df_ridership["stop_id"].combine_first(df_ridership["gtfs_stop_id"])

In [13]:
# df_ridership[df_ridership["stop_id_comb"].isna()]["organization_name"].unique()

**OmniTrans has stop names that are not matched in any stop name in GTFS.**

In [10]:
df_ridership[df_ridership["stop_id_comb"].isna()]

,Unnamed: 0,record_id,dataset_id,organization_name,service_name,route_id,route_name,direction,stop_id,stop_name,...,stop_lon,avg_boardings,avg_alightings,avg_ridership,day_type,agg_basis,start_date,end_date,gtfs_stop_id,stop_id_comb
1387166,1387166,D6F2B598A671C4EB,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,2ND @ F ST,...,NaN,2.572603,0.619178,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
1387167,1387167,8C772D5F9586B138,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,2ND @ G ST,...,NaN,1.410959,7.701370,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
1387189,1387189,E54CF08B65329762,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,EAST VALLEY,...,NaN,4.180822,3.473973,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
1387190,1387190,9A1C62A9E262B5E6,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,GILBERT @ KENWOOD,...,NaN,0.709589,1.536986,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
1387202,1387202,344DDD5B8D0F9084,40F5F343F0BD5850,OmniTrans,OmniTrans,1,NaN,NaN,<NA>,MT VERNON @ JOHNSON,...,NaN,5.550685,18.405479,NaN,all,fiscal year,2023-07-01,2024-06-30,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1391988,1391988,7911592B1D59A063,40F5F343F0BD5850,OmniTrans,OmniTrans,215,NaN,NaN,<NA>,EAST VALLEY,...,NaN,0.520548,0.416438,NaN,all,fiscal year,2025-07-01,2026-06-30,<NA>,<NA>
1391990,1391990,2D3AB0813C064DDC,40F5F343F0BD5850,OmniTrans,OmniTrans,215,NaN,NaN,<NA>,MT VERNON @ CENTREPOINTE,...,NaN,16.873973,17.495890,NaN,all,fiscal year,2025-07-01,2026-06-30,<NA>,<NA>
1391993,1391993,4434F9B74EB665B2,40F5F343F0BD5850,OmniTrans,OmniTrans,290,NaN,NaN,<NA>,ARROWHEAD MEDICAL CENTER,...,NaN,0.958904,0.843836,NaN,all,fiscal year,2025-07-01,2026-06-30,<NA>,<NA>
1391994,1391994,2105316659A4E028,40F5F343F0BD5850,OmniTrans,OmniTrans,290,NaN,NaN,<NA>,EAST VALLEY,...,NaN,0.063014,0.057534,NaN,all,fiscal year,2025-07-01,2026-06-30,<NA>,<NA>


### Handle OmniTrans Stop Name Format Inconsistency

In [14]:
omni_mask = df_ridership["organization_name"] == "OmniTrans"
t_df_omni = df_ridership.loc[omni_mask, ["stop_id_comb", "stop_name"]].copy()
t_df_omni["_orig_index"] = t_df_omni.index

In [15]:
street_types = ["street", "st", "avenue", "ave", "road", "rd", "boulevard", "blvd", "drive", "dr",
                "lane", "ln", "court", "ct", "place", "pl", "highway", "hwy", "way", "parkway", "pkwy"]

street_type_regex = r"\b(?:" + "|".join(street_types) + r")\b"

In [16]:
t_df_omni["stop_name_norm"] = t_df_omni["stop_name"].astype("string").str.lower().str.replace(r"\s+", " ", regex=True).str.strip()
# detect presence of any street type
t_df_omni["_has_street_type"] = t_df_omni["stop_name_norm"].str.contains(street_type_regex, regex=True, na=False)

t_df_omni["_stop_name_length"] = t_df_omni["stop_name_norm"].str.len()
# rank stop name variants within each stop id
canonical = t_df_omni.sort_values(by=["_has_street_type", "_stop_name_length"],
                                 ascending=[False, False]) \
                    .groupby("stop_id_comb", as_index=False).first()[["stop_id_comb", "stop_name"]].rename(columns={"stop_name": "canonical_stop_name"})
t_df_omni = t_df_omni.merge(canonical, on="stop_id_comb", how="left")
canonical.head()

,stop_id_comb,canonical_stop_name
0,1,ARROWHEAD REG MED CTR
1,10,Pacific High School
2,100,Foothill @ Riverside Ave
3,104,5th @ Medical Center
4,105,Merrill @ Riverside


In [17]:
df_ridership.loc[t_df_omni["_orig_index"], "stop_name"] = t_df_omni["canonical_stop_name"].values

### Agg by Stop ID and Stop Name

use stop_id_comb or gtfs_stop_id

In [28]:
df_agg = df_ridership.groupby(["dataset_id", "organization_name", "service_name", "gtfs_stop_id", "stop_name",
                               "stop_lat", "stop_lon", "day_type", "daily_ridership_basis", "start_date", "end_date"], as_index=False) \
                    .agg(daily_total_ridership = ("daily_total_ridership", lambda x: x.sum(min_count=1)),
                         daily_alightings = ("daily_alightings", lambda x: x.sum(min_count=1)),
                         daily_boardings = ("daily_boardings", lambda x: x.sum(min_count=1))).reset_index()

# drop index col
df_agg.drop(columns="index", inplace=True)

df_agg.head()

,dataset_id,organization_name,service_name,gtfs_stop_id,stop_name,stop_lat,stop_lon,day_type,daily_ridership_basis,start_date,end_date,daily_total_ridership,daily_alightings,daily_boardings
0,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,holiday,reported_daily,2024-10-14,2024-10-14,NaN,3666.0,3735.0
1,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,holiday,reported_daily,2024-11-11,2024-11-11,NaN,2301.0,2540.0
2,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,holiday,reported_daily,2024-11-28,2024-11-28,NaN,976.0,1021.0
3,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,holiday,reported_daily,2024-11-29,2024-11-29,NaN,1883.0,1971.0
4,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,holiday,reported_daily,2024-12-24,2024-12-24,NaN,1931.0,1940.0


In [29]:
t_df = df_agg.groupby(["dataset_id", "organization_name", "service_name", "gtfs_stop_id", "stop_name", "day_type", "start_date", "end_date"]).size().reset_index(name="row_count").sort_values(by="row_count", ascending=False)
t_df.head(10)

,dataset_id,organization_name,service_name,gtfs_stop_id,stop_name,day_type,start_date,end_date,row_count
0,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,holiday,2024-10-14,2024-10-14,1
793064,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-08-19,2025-08-19,1
793044,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-07-22,2025-07-22,1
793043,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-07-21,2025-07-21,1
793042,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-07-18,2025-07-18,1
793041,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-07-17,2025-07-17,1
793040,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-07-16,2025-07-16,1
793039,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-07-15,2025-07-15,1
793038,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-07-14,2025-07-14,1
793037,9B106785FD780293,City of Fresno,Fresno Area Express,1538.0,SW CEDAR - FLORENCE,weekday,2025-07-11,2025-07-11,1


In [30]:
t_df[t_df["row_count"] > 1]["organization_name"].unique()

array([], dtype=object)

In [31]:
t_df[t_df["row_count"] > 1].groupby(by=["organization_name", "service_name", "day_type", "start_date", "end_date"])["gtfs_stop_id"].nunique().reset_index()

,organization_name,service_name,day_type,start_date,end_date,gtfs_stop_id


#### Big Blue Bus

In [32]:
# df_big_blue_bus = df_agg[df_agg["organization_name"] == "City of Santa Monica"]

In [20]:
# df_big_blue_bus[(df_big_blue_bus["gtfs_stop_id"] == "585") & (df_big_blue_bus["start_date"] == "2024-08-01")]

#### Santa Cruz Metro

In [21]:
# t_df[(t_df["service_name"] == "Santa Cruz METRO") & (t_df["row_count"] > 1)]

**Note: The duplicates in SamTrans is the stop id that is not in GTFS - and in ridership data, a stop id can have more than one pair of lat/lon. Looks like it is vehicle position. So we use GTFS lat/lon mapped in using stop id for the aggregation.**

In [22]:
# df_omnitrans = df_agg[df_agg["organization_name"] == "OmniTrans"]
# df_omnitrans[(df_omnitrans["stop_id_comb"] == '6021') & (df_omnitrans["start_date"] == '2025-07-01')]

In [23]:
# df_omnitrans[(df_omnitrans["stop_id_comb"] == '5488') & (df_omnitrans["start_date"] == '2025-07-01')]

In [24]:
# df_omnitrans[(df_omnitrans["stop_id_comb"] == '7288') & (df_omnitrans["start_date"] == '2025-07-01')]

In [25]:
# t_df_omnitrans = df_omnitrans.groupby("stop_id_comb")["stop_name"].nunique().reset_index(name="num_stop_name")
# id_list_omnitrans = t_df_omnitrans[t_df_omnitrans["num_stop_name"] > 1]["stop_id_comb"].unique().tolist()

In [26]:
# df_omnitrans[(df_omnitrans["stop_id_comb"].isin(id_list_omnitrans))]

#### Add key

In [33]:
def generate_key(*args, len=16, sep="|"):
    """Generate short reproducible hash ID based on input string"""
    key = sep.join(str(arg) for arg in args if arg is not None).encode("utf-8")
    return hashlib.sha1(key).hexdigest()[:len].upper()

In [34]:
df_agg = df_agg.rename(columns={"gtfs_stop_id": "stop_id"})

In [35]:
df_agg["record_id"] = df_agg.apply(lambda row: generate_key(row["organization_name"],
                                                           row["service_name"],
                                                           row["stop_id"],
                                                           row["stop_name"],
                                                           row["day_type"],
                                                           row["start_date"]
                                                          ), axis=1, result_type="expand")

In [36]:
# check duplicates
t_df_agg = df_agg.groupby("record_id").size().reset_index(name="count")
t_df_agg[t_df_agg["count"] > 1]

,record_id,count


In [37]:
df_agg = df_agg[["record_id", "dataset_id", "organization_name", "service_name", "stop_id", "stop_name", "stop_lat", "stop_lon", 
                 "daily_alightings", "daily_boardings", "daily_total_ridership", "start_date", "end_date", "daily_ridership_basis", "day_type"]]

#### Export

In [22]:
today = date.today().strftime("%Y-%m-%d")
df_agg.to_csv(f"fact_ridership_output/fct_ridership_{today}.csv", index=False) # save a copy

In [23]:
df_agg.head()

,record_id,dataset_id,organization_name,service_name,stop_id,stop_name,stop_lat,stop_lon,avg_alightings,avg_boardings,avg_ridership,start_date,end_date,daily_ridership_basis,day_type
0,947E32A15D31E288,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,3666.0,3735.0,NaN,2024-10-14,2024-10-14,reported_daily,holiday
1,50A204C99EF493D2,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,2301.0,2540.0,NaN,2024-11-11,2024-11-11,reported_daily,holiday
2,A6B375D418BC5853,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,976.0,1021.0,NaN,2024-11-28,2024-11-28,reported_daily,holiday
3,0A853373D96249B8,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,1883.0,1971.0,NaN,2024-11-29,2024-11-29,reported_daily,holiday
4,6D6FA33361F1AFA3,011CF30F49575609,San Francisco Bay Area Rapid Transit District,Bay Area Rapid Transit,900109,12th Street / Oakland City Center,37.803482,-122.27163,1931.0,1940.0,NaN,2024-12-24,2024-12-24,reported_daily,holiday
